In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
search_key = os.environ["AZURE_SEARCH_ADMIN_KEY"]

We need "chain of thought" style agnet: https://blog.langchain.dev/planning-agents/

**Graph State**: In LangGraph, every node updates a shared graph state. The state is the input to any node whenever it is invoked.

Below, we will define a state dict to contain the task, plan, steps, and other variables.

In [37]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

from langchain.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)

## Embedding/vectorisation

In [3]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    chunk_size=1 
)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=search_endpoint,
    azure_search_key=search_key,
    index_name="boardai03",
    embedding_function=embeddings.embed_query,
)

[llm]

In [4]:
llm = AzureChatOpenAI(
    deployment_name="gpt-4",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    # temperature=0
)

In [47]:
retriever = vector_store.as_retriever(search_key="hybrid", search_kwargs={"k": 2})


prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)

In [7]:
while True:
    query = input("User: ")
    if query == "exit":
        break
    print("query:",query , "\nanswer: ", rag_chain.invoke(query))

query: how do i create a board paper? 
 answer:  To create a board paper, you first need to establish an agenda based on approved topics such as performance, operation, and profit, and email your board members (NEC) about it. The board members will then write the paper. Once you receive their responses and compile them into a paper, you can make necessary amendments based on feedback before circulating the finalized version to all board members.


## Planner
https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb?ref=blog.langchain.dev

In [43]:
prompt_planner = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:

(1) RAG+LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general
world knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

(2) sendEmail_chase_gatherPaper[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. 
Input will be a list of board members and agendas.

(3) compilePaper[input]:Worker that add all the separate paper into one board paper. input will be multiple board paper pieces from board members.

(4) circulatePaper[input]: Worker that send compiled board paper to board members and get feed back and fix. Input will be a board paper. 

(5) sendReminderEmail[input]: xyz

For example,
Task: execute this: To create a board paper, first, based on an approved agenda, the secretary sends an email to the NEC (or board members). The NEC then writes the paper. 
After the paper is written, the secretary compile, circulates and distributes the paper to the board members.


Plan: Find out what is approved agenda. #E1 = RAG+LLM[what is approved agenda]

Plan: Find out who are the NEC or board members. #E2 = RAG+LLM[who are the NEC or board members]

Plan: the secretary sends an email to the NEC (or board members).The NEC writes the paper. #E3 = sendEmail_chase_gatherPaper[#E2]

Plan: compilePaper #E4 = compilePaper[#E3] 

Plan: After the paper is written, the secretary circulates and distributes the paper to the board members. #E5 = circulatePaper[#E4]


Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.
\nQuestion: {question}
\nContext: {context}
\nAnswer:""" 


In [23]:
# rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [48]:
p_prompt = ChatPromptTemplate(input_variables=['context', 'question'], 
                            metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, 
                            messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=prompt_planner))])

In [49]:
rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| p_prompt
| llm
| StrOutputParser()
)

In [50]:
res = rag_chain.invoke(query)


'Plan: Determine what the agendas are for the board paper. #E1 = RAG+LLM[what are the agendas for the board paper]\n\nPlan: Identify the NEC or board members who will be contributing to the board paper. #E2 = RAG+LLM[who are the NEC or board members]\n\nPlan: Send an email to the NEC or board members asking them to write their sections of the board paper based on the determined agendas. Chase up any members who have not responded and compile the responses. #E3 = sendEmail_chase_gatherPaper[#E2, #E1]\n\nPlan: Compile all the received board paper pieces into one comprehensive board paper. #E4 = compilePaper[#E3] \n\nPlan: Show the compiled board paper to the person in charge and ask if they want to make any fixes. If they say yes, make the specified fixes. #E5 = RAG+LLM[Do you want to fix anything in the compiled board paper?]\n\nPlan: If no fixes are needed or once all fixes are made, circulate the final version of the board paper to all board members. #E6 = circulatePaper[#E5] \n\nPlan

In [51]:
print(res)

Plan: Determine what the agendas are for the board paper. #E1 = RAG+LLM[what are the agendas for the board paper]

Plan: Identify the NEC or board members who will be contributing to the board paper. #E2 = RAG+LLM[who are the NEC or board members]

Plan: Send an email to the NEC or board members asking them to write their sections of the board paper based on the determined agendas. Chase up any members who have not responded and compile the responses. #E3 = sendEmail_chase_gatherPaper[#E2, #E1]

Plan: Compile all the received board paper pieces into one comprehensive board paper. #E4 = compilePaper[#E3] 

Plan: Show the compiled board paper to the person in charge and ask if they want to make any fixes. If they say yes, make the specified fixes. #E5 = RAG+LLM[Do you want to fix anything in the compiled board paper?]

Plan: If no fixes are needed or once all fixes are made, circulate the final version of the board paper to all board members. #E6 = circulatePaper[#E5] 

Plan: Confirm tha